In [1]:
# Uncomment to run the notebook in Colab
# ! pip install -q "wax-ml[complete]@git+https://github.com/eserie/wax-ml.git"
# ! pip install -q --upgrade jax jaxlib==0.1.67+cuda111 -f https://storage.googleapis.com/jax-releases/jax_releases.html

In [2]:
# check available devices
import jax

In [3]:
print("jax backend {}".format(jax.lib.xla_bridge.get_backend().platform))
jax.devices()

jax backend cpu


[CpuDevice(id=0)]

# 🎛 The 3-steps workflow 🎛

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eserie/wax-ml/blob/main/docs/notebooks/04_The_three_steps_workflow.ipynb)

It is already very useful to be able to execute a JAX function on a dataframe in a single work step
and with a single command line thanks to WAX-ML accessors.

The 1-step WAX-ML's stream API works like that:
```python
<data-container>.stream(...).apply(...)
```

But this is not optimal because, under the hood, there are mainly three costly steps:
- (1) (synchronize | data tracing | encode): make the data "JAX ready"
- (2) (compile | code tracing | execution): compile and optimize a function for XLA, execute it.
- (3) (format): convert data back to pandas/xarray/numpy format.

With the `wax.stream` primitives, it is quite easy to explicitly split the 1-step workflow
into a 3-step workflow.

This will allow the user to have full control over each step and iterate on each one.

It is actually very useful to iterate on step (2), the "calculation step" when
you are doing research.
You can then take full advantage of the JAX primitives, especially the `jit` primitive.

Let's illustrate how to reimplement WAX-ML EWMA yourself with the WAX-ML 3-step workflow.

## Imports

In [4]:
import haiku as hk
import numpy as onp
import pandas as pd
import xarray as xr

from wax.accessors import register_wax_accessors
from wax.compile import jit_init_apply
from wax.external.eagerpy import convert_to_tensors
from wax.format import format_dataframe
from wax.modules import EWMA
from wax.stream import tree_access_data
from wax.unroll import dynamic_unroll

register_wax_accessors()

ModuleNotFoundError: No module named 'optax'

## Performance on big dataframes

### Generate data

In [ ]:
T = 1.0e5
N = 1000

In [ ]:
T, N = map(int, (T, N))
dataframe = pd.DataFrame(
    onp.random.normal(size=(T, N)), index=pd.date_range("1970", periods=T, freq="s")
)

### pandas EWMA

In [ ]:
%%time
df_ewma_pandas = dataframe.ewm(alpha=1.0 / 10.0).mean()

### WAX-ML EWMA

In [ ]:
%%time
df_ewma_wax = dataframe.wax.ewm(alpha=1.0 / 10.0).mean()

It's a little faster, but not that much faster...

### WAX-ML EWMA (without format step)

Let's disable the final formatting step (the output is now in raw JAX format):

In [ ]:
%%time
df_ewma_wax_no_format = dataframe.wax.ewm(alpha=1.0 / 10.0, format_outputs=False).mean()
df_ewma_wax_no_format.block_until_ready()

In [ ]:
type(df_ewma_wax_no_format)

Let's check the device on which the calculation was performed (if you have GPU available, this should be `GpuDevice` otherwise it will be `CpuDevice`):

In [ ]:
df_ewma_wax_no_format.device()

Now we will see how to break down WAX-ML one-liners `<dataset>.ewm(...).mean()` or `<dataset>.stream(...).apply(...)` into 3 steps:
- a preparation step where we prepare JAX-ready data and functions.
- a processing step where we execute the JAX program
- a post-processing step where we format the results in pandas or xarray format.

### Generate data (in dataset format)

WAX-ML `Sream` object works on datasets.
So let's transform the `DataFrame` into a xarray `Dataset`:

In [ ]:
dataset = xr.DataArray(dataframe).to_dataset(name="dataarray")
del dataframe

## Step (1) (synchronize | data tracing | encode)

In this step,  WAX-ML do:
- "data tracing" : prepare the indices for fast access tin the JAX function `access_data`
- synchronize streams if there is multiple ones.
  This functionality have options : `freq`, `ffills`
- encode and convert data from numpy to JAX: use encoders for `datetimes64` and `string_`
  dtypes. Be aware that by default JAX works in float32
  (see [JAX's Common Gotchas](https://jax.readthedocs.io/en/latest/notebooks/Common_Gotchas_in_JAX.html#double-64bit-precision) to work in float64).

We have a function `Stream.prepare` that implement this Step (1).
It prepares a function that wraps the input function with the actual data and indices
in a pair of pure functions (`TransformedWithState` Haiku tuple).

In [ ]:
%%time
stream = dataset.wax.stream()

Define our custom function to be applied on a dict of arrays
having the same structure than the original dataset:

In [ ]:
def my_ewma_on_dataset(dataset):
    return EWMA(alpha=1.0 / 10.0, adjust=True)(dataset["dataarray"])

In [ ]:
transform_dataset, jxs = stream.prepare(dataset, my_ewma_on_dataset)

Let's definite the init parameters and state of the transformation we
will apply.

### Init params and state

In [ ]:
from wax.unroll import init_params_state

In [ ]:
rng = jax.random.PRNGKey(42)
params, state = init_params_state(transform_dataset, rng, jxs)

In [ ]:
params

In [ ]:
assert state["ewma"]["count"].shape == (N,)
assert state["ewma"]["mean"].shape == (N,)

## Step (2) (compile | code tracing | execution)

In this step we:
- prepare a pure function (with
  [Haiku's transform mechanism](https://dm-haiku.readthedocs.io/en/latest/api.html#haiku-transforms))
  Define a "transformation" function which:
    - access to the data
    - apply another transformation, here: EWMA

- compile it with `jax.jit`
- perform code tracing and execution (the last line):
    - Unroll the transformation on "steps" `xs` (a `np.arange` vector).

In [ ]:
rng = next(hk.PRNGSequence(42))
outputs, state = dynamic_unroll(transform_dataset, params, state, rng, False, jxs)

In [ ]:
outputs.device()

Once it has been compiled and "traced" by JAX, the function is much faster to execute:

In [ ]:
%%timeit
outputs, _ = dynamic_unroll(transform_dataset, params, state, rng, False, jxs)
_ = outputs.block_until_ready()

This is 3x faster than pandas implementation!

### Manually prepare the data and manage the device

In order to manage the device on which the computations take place,
we need to have even more control over the execution flow.
Instead of calling `stream.prepare` to build the `transform_dataset` function,
we can do it ourselves by :
- using the `stream.trace_dataset` function
- converting the numpy data in jax ourself
- puting the data on the device we want.

In [ ]:
np_data, np_index, xs = stream.trace_dataset(dataset)
jnp_data, jnp_index, jxs = convert_to_tensors((np_data, np_index, xs), "jax")

We explicitly set data on CPUs (the is not needed if you only have CPUs):

In [ ]:
from jax.tree_util import tree_leaves, tree_map

cpus = jax.devices("cpu")
jnp_data, jnp_index, jxs = tree_map(
    lambda x: jax.device_put(x, cpus[0]), (jnp_data, jnp_index, jxs)
)
print("data copied to CPU device.")

We have now "JAX-ready" data for later fast access.

Let's define the transformation that wrap the actual data and indices in a pair of
pure functions:

In [ ]:
@jit_init_apply
@hk.transform_with_state
def transform_dataset(step):
    dataset = tree_access_data(jnp_data, jnp_index, step)
    return EWMA(alpha=1.0 / 10.0, adjust=True)(dataset["dataarray"])

And we can call it as before:

In [ ]:
%%time
outputs, state = dynamic_unroll(transform_dataset, None, None, rng, False, jxs)
_ = outputs.block_until_ready()

In [ ]:
outputs.device()

## Step(3) (format)
Let's come back to pandas/xarray:

In [ ]:
%%time
y = format_dataframe(
    dataset.coords, onp.array(outputs), format_dims=dataset.dataarray.dims
)

It's quite slow (see WEP3 enhancement proposal).

## GPU execution

Let's look with execution on GPU

In [ ]:
try:
    gpus = jax.devices("gpu")
    jnp_data, jnp_index, jxs = tree_map(
        lambda x: jax.device_put(x, gpus[0]), (jnp_data, jnp_index, jxs)
    )
    print("data copied to GPU device.")
    GPU_AVAILABLE = True
except RuntimeError as err:
    print(err)
    GPU_AVAILABLE = False

Let's check that our data is on the GPUs:

In [ ]:
tree_leaves(jnp_data)[0].device()

In [ ]:
tree_leaves(jnp_index)[0].device()

In [ ]:
jxs.device()

In [ ]:
%%time
if GPU_AVAILABLE:
    rng = next(hk.PRNGSequence(42))
    outputs, state = dynamic_unroll(transform_dataset, None, None, rng, False, jxs)

Let's redefine our function `transform_dataset` by explicitly specify to `jax.jit` the `device` option.

In [ ]:
%%time
if GPU_AVAILABLE:

    @hk.transform_with_state
    def transform_dataset(step):
        dataset = tree_access_data(jnp_data, jnp_index, step)
        return EWMA(alpha=1.0 / 10.0, adjust=True)(dataset["dataarray"])

    transform_dataset = type(transform_dataset)(
        transform_dataset.init, jax.jit(transform_dataset.apply, device=gpus[0])
    )

    rng = next(hk.PRNGSequence(42))
    outputs, state = dynamic_unroll(transform_dataset, None, None, rng, False, jxs)

In [ ]:
outputs.device()

In [ ]:
%%timeit
if GPU_AVAILABLE:
    outputs, state = dynamic_unroll(transform_dataset, None, None, rng, False, jxs)
    _ = outputs.block_until_ready()